## <a id='toc1_1_'></a>[To use Fun_Upgrade.py, upgrade the script](#toc0_)

### <a id='toc1_1_1_'></a>[Functions to achieve:](#toc0_)
1. reduce ram used; 
2. reduce code simplicity by building new class; (class: Controller; Path; )
3. save more frequently
4. reduce test time

In [21]:
# magic lines that avoid re-start 
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [22]:
# Load modules
import pybamm as pb;import pandas as pd;import numpy as np;
import os, json,openpyxl,traceback,multiprocessing,scipy.optimize,sys,gc
import matplotlib.pyplot as plt;
import pickle,imageio,timeit,random,time, signal
from scipy.io import savemat,loadmat;
from pybamm import constants,exp;import matplotlib as mpl

In [23]:
str_path_0 = os.path.abspath(os.path.join(pb.__path__[0],'..'))
str_path_1 = os.path.abspath(
    os.path.join(str_path_0,"wip/Rio_Code/Fun_Upgrade"))
str_path_1

'/home/ruihe_li/EnvPBGEM_NC/PyBaMM-GEM/wip/Rio_Code/Fun_Upgrade'

In [24]:
# reduce variables that require define outside
On_HPC = False
if On_HPC:
    case_no = int(os.environ["PBS_ARRAY_INDEX"])
    Path_Input = "InputData/" 
    BasicPath_Save=os.getcwd() 
else:
    str_path_0 = os.path.abspath(os.path.join(pb.__path__[0],'..'))
    str_path_1 = os.path.abspath(
        os.path.join(str_path_0,"wip/Rio_Code/Fun_Upgrade"))
    sys.path.append(str_path_1) 
    
    case_no = 6
    Path_Input = os.path.expanduser(
        "~/EnvPBGEM_NC/SimSave/InputData/") # for Linux
    BasicPath_Save =  os.path.expanduser(
        "~/EnvPBGEM_NC/SimSave/P2_R9_Dim")
# import all functions 
from Fun_Upgrade import Run_One_Case
from Fun_Upgrade import *
from TaskResult import *
from TaskConfig import *
from Get_input import *

In [25]:
# define class for all inputs and customized settings:
# start to do configuration:
path_config = Path_config(
    On_HPC=On_HPC, Path_Input=Path_Input,
    BasicPath_Save=BasicPath_Save,
    purpose_i="Full_Exp1235_NC", case_no=case_no,
    rows_per_file= 1,Scan_end_end= 12,
    )
global_config = Global_config(
    On_HPC=On_HPC, Runshort=True, 
    Plot_Exp=True, Timeout=True, Return_Sol=True, 
    Check_Short_Time=True, R_from_GITT=True,
    fs=13, dpi=100, Re_No=0, Timelimit=int(3600*48),
    Timeout_text = 'I timed out'   )

exp_config = Exp_config()
para_config = Para_config()
model_config = Model_config()
expData_config = ExpData_config(cap_0 = 4.86491)
config = TaskConfig(
    path_config=path_config, global_config=global_config, 
    exp_config=exp_config,para_config=para_config,
    model_config=model_config,expData_config=expData_config)
# Load input file

# normal version
# Para_dict_list = load_combinations_from_csv(config.path_config.Para_file) 

# ! For Dan to test:
path_csv_special_for_test = str_path_1 + "/Bundle_6.csv"
Para_dict_list = load_combinations_from_csv(path_csv_special_for_test) 
pool_no = len(Para_dict_list) # do parallel computing if needed

Total scan case is 1


In [26]:
Para_dict_list[0]['Mesh list'] = '[5,5,5,60,20]'
Para_dict_list[0]

{'Scan No': 6,
 'Exp No.': 2,
 'Ageing temperature': 40,
 'Mesh list': '[5,5,5,60,20]',
 'Inner SEI lithium interstitial diffusivity [m2.s-1]': 9.81e-19,
 'Dead lithium decay constant [s-1]': 1e-07,
 'Lithium plating kinetic rate constant [m.s-1]': 1e-10,
 'Negative electrode LAM constant proportional term [s-1]': 2.84e-09,
 'Positive electrode LAM constant proportional term [s-1]': 2.98e-18,
 'Negative electrode cracking rate': 5.29e-25,
 'Outer SEI partial molar volume [m3.mol-1]': 5.22e-05,
 'SEI growth activation energy [J.mol-1]': 5000.0,
 'Negative cracking growth activation energy [J.mol-1]': 0.0,
 'Negative electrode diffusivity activation energy [J.mol-1]': 20000.0,
 'Positive electrode diffusivity activation energy [J.mol-1]': 12000.0,
 'Initial electrolyte excessive amount ratio': 1.0,
 'Cycles within RPT': 1,
 'RPT temperature': 25,
 'Para_Set': 'OKane2023',
 'Model option': "{'SEI': 'interstitial-diffusion limited', 'SEI on cracks': 'true', 'lithium plating': 'partially re

In [27]:

# Assuming we only do the first one:
config.Get_Para_dict_i(Para_dict_list[0])

# config.para_config.Para_dict_i['Exp No.'] = 8

config.Get_Exp_config()
config.Initialize_exp_text()
config.Get_Exp_Pack()

Aging T is 40degC
Finish reading Experiment!


In [28]:
Para_dict_list[0]

{'Scan No': 6,
 'Exp No.': 2,
 'Ageing temperature': 40,
 'Mesh list': '[5,5,5,60,20]',
 'Inner SEI lithium interstitial diffusivity [m2.s-1]': 9.81e-19,
 'Dead lithium decay constant [s-1]': 1e-07,
 'Lithium plating kinetic rate constant [m.s-1]': 1e-10,
 'Negative electrode LAM constant proportional term [s-1]': 2.84e-09,
 'Positive electrode LAM constant proportional term [s-1]': 2.98e-18,
 'Negative electrode cracking rate': 5.29e-25,
 'Outer SEI partial molar volume [m3.mol-1]': 5.22e-05,
 'SEI growth activation energy [J.mol-1]': 5000.0,
 'Negative cracking growth activation energy [J.mol-1]': 0.0,
 'Negative electrode diffusivity activation energy [J.mol-1]': 20000.0,
 'Positive electrode diffusivity activation energy [J.mol-1]': 12000.0,
 'Initial electrolyte excessive amount ratio': 1.0,
 'Cycles within RPT': 1,
 'RPT temperature': 25,
 'Para_Set': 'OKane2023',
 'Model option': "{'SEI': 'interstitial-diffusion limited', 'SEI on cracks': 'true', 'lithium plating': 'partially re

### Run the model

In [29]:
# Run the model, assuming only one case:
if config.global_config.Re_No == 0:
    task_result, config = Run_One_Case(config) 
elif config.global_config.Re_No > 0:
    pass


Start Now! Scan 6 Re 0
Has Overwritten Initial outer SEI thickness [m] to be 
        9.23e-09 and Negative electrode 
        porosity to be 0.235 to account for initial 
        capacity loss of 0.135 Ah
Initial electrolyte amount is  5.236163896909271 mL
Scan 6 Re 0: Spent 1.210 ms on Initialization, DryOut = On



Succeed to run break in for the 0th time with perturbation of 0.00e+00Ah

Scan 6 Re 0: Finish break-in cycle within 51.033 s
Scan 6 Re 0: Finish post-process for break-in cycle within 991.951 ms
Electrolyte is being squeezed out, check plated lithium (active and dead)
Succeed to run the ageing set for 1 cycles the 1th time
Scan 6 Re 0: Finish No.
              0 
              ageing cycles within 10.941 s
Scan 6 Re 0: Finish post-process for 
              No. 1 
              ageing cycles within 117.892 ms
Electrolyte is being squeezed out, check plated lithium (active and dead)
Succeed to run RPT for the 1th time
Scan 6 Re 0: Finish No."
            1"
            RPT cycles within 36.685 s
Scan 6 Re 0: Finish post-process for No.
            1 
            RPT cycles within 2.020 s
Scan 6 Re 0: Getting extra variables within 1.933 ms
Scan 6 Re 0: Finish all plots within 7.113 s
Electrolyte is being squeezed out, check plated lithium (active and dead)
Scan 6 Re 0: Save Early for R